In [ ]:
!pip install openai
!pip install pinecone-client
!pip install -U transformers
!pip install accelerate

In [ ]:
from google.colab import userdata
import openai

openai_api_key = userdata.get('openai-secret')
pinecone_api_key = userdata.get('pinecone-api-key')
hface_read_token = userdata.get('hface-read-token')

#1. Read the data and create dataset with embeddings

In [ ]:
from openai import OpenAI

def get_embeddings(text):
    client = OpenAI(api_key=openai_api_key)
    response = client.embeddings.create(input=text,model="text-embedding-ada-002")
    return response.data[0].embedding

In [ ]:
import pandas as pd
import uuid
import json

def create_dataframe(texts):
    data = {
        "id": [],
        "embeddings_vector": [],
        "metadata": []
    }

    for i, text in enumerate(texts):
        data["id"].append(str(uuid.uuid4()))
        data["embeddings_vector"].append(get_embeddings(text))
        data["metadata"].append(json.dumps({"chunk": i, "text": text}))

    return pd.DataFrame(data)

In [ ]:
with open('/content/asari-data-en.txt', 'r') as asariFile:
    content = asariFile.read()

texts = content.split("<br/>")
texts
len(texts)

9

In [ ]:
df = create_dataframe(texts)
df.head()

,id,embeddings_vector,metadata
0,0f226747-0357-4106-9d83-dc969181b2e5,"[0.0014285765355452895, -0.023303022608160973,...","{""chunk"": 0, ""text"": ""ASARI.\nNative to the pl..."
1,f7755a10-dd73-45fd-8c6a-d0b88b075fe7,"[-8.967203757492825e-05, -0.00518527626991272,...","{""chunk"": 1, ""text"": ""\nBiology\nLauna T'Saphr..."
2,b07d133c-64a0-40d8-ad0f-664e0990b18c,"[0.022635992616415024, -0.018505915999412537, ...","{""chunk"": 2, ""text"": ""\nHISTORY\nThe Asari wer..."
3,01fb09f5-3290-4ed6-b815-d0c165fa061a,"[0.000832043937407434, -0.01641482673585415, 0...","{""chunk"": 3, ""text"": ""\nCULTURE\nDue to their ..."
4,71fb12a8-7fd5-41ea-8950-76108aadd6f3,"[0.013306813314557076, 0.00113303295802325, 4....","{""chunk"": 4, ""text"": ""\nRELIGION\nThe mainstre..."


#2. Load the embeddings into Pinecone

In [ ]:
from pinecone import Pinecone

pc = Pinecone(api_key=pinecone_api_key)
index = pc.Index("tech16llm01")
index

In [ ]:
import json

vectors = []
for i, row in df.iterrows():
  vectors.append({
      "id": row['id'],
      "values": row['embeddings_vector'],
      "metadata": json.loads(row['metadata'])
  });

index.upsert(vectors, namespace='mass-effect')

{'upserted_count': 9}

#3. Definition of function to retrieve the data from Pinecone

In [ ]:
def get_rag_relevant_info(textQuery):
  query_embeddings = get_embeddings(textQuery)
  query_result = index.query(vector=query_embeddings, top_k=3, include_metadata=True, namespace = 'mass-effect')
  relevant_results = [result for result in query_result['matches'] if result.score >= 0.8]

  return relevant_results

#4. Definition of function to create the prompt to augment the query

In [ ]:
def get_augmented_query(textQuery):
  relevant_results = get_rag_relevant_info(textQuery)

  print(f"Relevant results for {textQuery}: {relevant_results}")

  augmented_query = textQuery
  if len(relevant_results) > 0:
    contexts = [item['metadata']['text'] for item in relevant_results]
    augmented_query =  "Query:" + textQuery + "\n\n" + "Continue to answer the query by using the next information:\n\n" + "\n\n---\n\n".join(contexts)

    print(f"Augmented query: {augmented_query}")

  return augmented_query, relevant_results

#5. Creation of GEMMA model and tokenizer

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("google/gemma-2b-it", token=hface_read_token)
model = AutoModelForCausalLM.from_pretrained("google/gemma-2b-it", device_map="auto", token=hface_read_token)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

#6.1 Test case 1: Ask for any notable Asari with no access to the vector storage. The model do not answer correctly

In [ ]:
query1 = 'Tell me the name of any notable Asari'

input1_ids = tokenizer(query1, return_tensors="pt").to("cuda")
response1 = model.generate(**input1_ids, max_new_tokens=500)

print(tokenizer.decode(response1[0]))

<bos>Tell me the name of any notable Asari character in the Halo universe.

Sure, here's a list of notable Asari characters in the Halo universe:

* Cortana
* Miranda Keyes
* Arbiter
* Chieftain
* Master Chief<eos>


#6.2 Test case 2. Get the augmented query and create a prompt to provide the context data about Asari to the Gemma LLM. The model answers correctly.

In [ ]:
augmented_query, relevant_results = get_augmented_query(query1)

print(relevant_results)
print('-' * 80)
print(augmented_query)
print('-' * 80)

Relevant results for Tell me the name of any notable Asari: [{'id': 'd48099fc-6708-44e1-b449-0374b6b06527',
 'metadata': {'chunk': 0.0,
              'text': 'ASARI.\n'
                      'Native to the planet Thessia, the asari are often '
                      'considered the most powerful and respected race in the '
                      'entire galaxy. This is due in part to the fact that the '
                      'Asari were the first race, after the Protheans, to '
                      'achieve interstellar flight and discover the Citadel.\n'
                      '\n'
                      'This breed is mono-gender (clearly female in '
                      'appearance). The Asari are known for their elegance and '
                      'their talent for diplomacy and biotics. They have '
                      'extreme longevity and their unique physiology allows '
                      'them to reproduce with a partner of any gender and/or '
                      'specie

In [ ]:
input2_ids = tokenizer(augmented_query, return_tensors="pt").to("cuda")
response2 = model.generate(**input2_ids, max_new_tokens=500)

print(tokenizer.decode(response2[0]))

<bos>Query:Tell me the name of any notable Asari

Continue to answer the query by using the next information:

ASARI.
Native to the planet Thessia, the asari are often considered the most powerful and respected race in the entire galaxy. This is due in part to the fact that the Asari were the first race, after the Protheans, to achieve interstellar flight and discover the Citadel.

This breed is mono-gender (clearly female in appearance). The Asari are known for their elegance and their talent for diplomacy and biotics. They have extreme longevity and their unique physiology allows them to reproduce with a partner of any gender and/or species.

The Asari were instrumental in the proposal and founding of the Citadel Council, and have been the core of galactic society ever since.


---


HISTORY
The Asari were the first race to achieve space flight and discover mass relays. When they came across the citadel, around 580 BC, they also encountered the caretakers, whose silent assistance all